![banner](../static/imgs/MagicMovieMachine_banner.png)
# 자신만의 영화 추천 시스템 만들기

이제 AWS 계정에 자신만의 영화 추천 시스템을 만들어 보겠습니다. 이 Jupyter 노트북에서
Magic Movie Machine처럼 추천을 생성하는 추천 시스템을 만드는 방법을 참조할 수 있습니다.

먼저 [MovieLens 프로젝트](https://grouplens.org/datasets/movielens/) 데이터 세트에서 수집된 영화 데이터를 다운로드하고 수정한 후 Amazon Personalize로 가져옵니다. 그런 다음 Amazon Personalize가 이 데이터를 사용하여 *Top picks for you* (고객님을 위한 탑 픽) 사용 사례와 *More like X* (X와 유사한 다른 상품) 사용 사례의 추천을 하나씩 생성하게 합니다.

이 두 사용 사례는 추천을 생성하는 사전 구성된 사용 사례입니다. 내부적으로, Amazon Personalize는 각 추천 시스템에 대해 여러 사용 사례별 모델을 훈련합니다. *Top picks for you* (고객님을 위한 탑 픽) 사용 사례는 Magic Movie Machine에서 보았던 Top 추천과 유사한 추천을 생성합니다. *More like X* (X와 유사한 다른 상품)는 *Hidden gems* 추천을 단순화한 버전으로, 비슷한 영화에 대한 추천을 생성하지만 덜 알려진 영화를 추천하는 것은 아닙니다.

마지막으로, Magic Movie Machine에서 보았던 것처럼 영화 추천을 받게 됩니다.

## 튜토리얼 비용 및 소요 시간
이 튜토리얼은 완료하는 데 2시간 정도 걸릴 수 있습니다. 사용하는 AWS 계정에 따라 약간의 비용이 발생할 수 있습니다. AWS 프리 티어를 사용하는 경우, 시작하는 당일에 튜토리얼을 완료하고 완료 시 리소스를 삭제하면 이 같은 비용이 거의 발생하지 않습니다. 요금과 가격의 전체 목록은 [Amazon Personalize 요금](https://aws.amazon.com/personalize/pricing/)을 참조하세요.

불필요한 요금이 발생하지 않도록 하려면 이 리포지토리에 있는 `Clean_Up_Resources.ipynb` 노트북을 실행하세요.
# 이 노트북을 사용하는 방법

코드는 아래와 같이 셀로 구분됩니다. 이 페이지 맨 위에 삼각형 모양의 실행(Run) 버튼이 있습니다. 이 버튼을 클릭하여 각 셀을 실행하고 다음 셀로 이동하거나, 셀에 있는 동안 `Shift` + `Enter`를 눌러 셀을 실행한 후 다음 셀로 이동할 수 있습니다.

셀이 실행되면, 셀이 실행되는 동안에는 옆의 라인에 `*`가 표시되며, 셀 내의 모든 코드 실행을 완료하고 나면 이 기호가 실행을 완료한 마지막 셀을 나타내는 숫자로 업데이트됩니다.

아래의 지침에 따라 셀을 실행하여, 사례에 최적화된 추천을 사용해 Amazon Personalize를 시작하세요.

## 가져오기
Python은 다양한 라이브러리와 함께 제공됩니다. 이러한 라이브러리와 함께, 핵심 데이터 과학 도구인 [boto3](https://aws.amazon.com/sdk-for-python/)(Python용 AWS SDK) 및 [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)와 같은 추가 패키지도 가져와야 합니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

그런 다음 사용자 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 훈련 데이터 다운로드, 준비 및 업로드
이 노트북에서는 [MovieLens 프로젝트](https://grouplens.org/datasets/movielens/)에서 수집된 상호 작용 데이터를 사용합니다. 아래의 코드 줄을 실행하여 MovieLens 데이터의 최신 버전을 다운로드하고 노트북으로 가져온 다음 빠르게 검사합니다.

### 데이터 세트 다운로드 및 탐색

In [9]:
!wget -N https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

--2022-04-09 16:48:16--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org... 128.101.65.152
Connecting to files.grouplens.org|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File 'ml-latest-small.zip' not modified on server. Omitting download.

Archive:  ml-latest-small.zip
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [5]:
!ls ml-latest-small

README.txt  links.csv   movies.csv  ratings.csv tags.csv


In [11]:
!pygmentize ml-latest-small/README.txt

Summary

This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from [MovieLens](http://movielens.org), a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.

Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.

The data are contained in the files `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. More details about the contents and use of all these files follows.

This is a *development* dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available *benchmark* datasets if that is your intent.

This and other GroupLens data sets are publicly available for down

In [7]:
interactions_data = pd.read_csv('./ml-latest-small/ratings.csv')
pd.set_option('display.max_rows', 5)
interactions_data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
...,...,...,...,...
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [8]:
interactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## 데이터 준비

### 상호 작용 데이터
보시다시피, 상호 작용 데이터에는 UserID, ItemID, 평점 및 타임스탬프가 포함됩니다. 데이터를 가져오려면 몇 가지 단계를 완료해야 합니다.

아래의 코드를 실행하여 순위가 낮은 항목을 제거하고, 데이터를 가져오기 전에 평점 열을 제거합니다. 또한 각 상호 작용에 대해 시청 이벤트 유형을 사용하여 EVENT_TYPE 열을 추가합니다.

In [ ]:
interactions_data = interactions_data[interactions_data['rating'] > 3]                # Keep only movies rated higher than 3 out of 5.
interactions_data = interactions_data[['userId', 'movieId', 'timestamp']]
interactions_data.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True)
interactions_data['EVENT_TYPE']='watch' #Adds an EVENT_TYPE column and an event type of "watch" for each interaction.
interactions_data.head()

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

### 항목 메타데이터

최상의 Magic Movie Machine을 만들려면 각 영화에 대한 데이터도 업로드해야 합니다. 항목 데이터 파일을 열고 처음 몇 행을 살펴봅니다.

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)

In [ ]:
items_data = pd.read_csv('./ml-latest-small/movies.csv')
items_data.head(5)
items_data.info()

In [ ]:
items_data['year'] = items_data['title'].str.extract('.*\((.*)\).*',expand = False)
items_data.head(5)

각 영화의 실제 생성 타임스탬프를 알 수 없으므로, 다음에서는 생성 타임스탬프로 최근의 날짜를 추가합니다.

In [ ]:
ts= datetime.datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

In [ ]:
items_data["CREATION_TIMESTAMP"] = ts
items_data

In [ ]:
# removing the title
items_data.drop(columns="title", inplace = True)

# renaming the columns to match schema
items_data.rename(columns = { 'movieId':'ITEM_ID', 'genres':'GENRES',
                              'year':'YEAR'}, inplace = True)
items_data

### 사용자 메타데이터

Magic Movie Machine에서 사용자 메타데이터를 기반으로 개인화를 제공하기 위해서는 사용자에 대한 데이터를 가져와야 합니다. MovieLens 데이터 세트에는 사용자 메타데이터가 없으므로 다음과 같이 영화와 상호 작용하는 각 사용자에 대해 가짜 메타데이터를 만들 수 있습니다.

In [ ]:
# get user ids from the interaction dataset

user_ids = interactions_data['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]=user_ids
user_data

#### 메타데이터 추가
현재 데이터 세트에는 추가 사용자 정보가 없습니다. 이 예에서는 남성일 확률과 여성일 확률을 동일하게 적용하여 사용자에게 임의로 성별을 할당합니다. 이와 같은 속성은 추천을 보다 개인화하는 데 효과적일 수 있습니다.

In [ ]:
possible_genders = ['female', 'male']
random = np.random.choice(possible_genders, len(user_data.index), p=[0.5, 0.5])
user_data["GENDER"] = random
user_data

## S3 버킷 및 IAM 역할 구성

지금까지 이 Jupyter 노트북을 실행하는 SageMaker 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다. 아래의 문자열로 리전을 정의하면 됩니다.

In [ ]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled this should be the same region as the Amazon Personalize resources
# region = "us-east-1"


In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalizemanagedvod"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except:
    print("Bucket already exists. Using bucket", bucket_name)

### S3에 데이터 업로드
이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다.

In [ ]:
interactions_filename = "interactions.csv"
interactions_data.to_csv(interactions_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

items_filename = "items.csv"
items_data.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

user_filename = "users.csv"
user_data.to_csv(user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_filename).upload_file(user_filename)

## S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

참고: 이 노트북에서 코드를 실행하는 데 사용 중인 역할에 S3 버킷 정책을 수정하는 데 필요한 권한이 있는지 확인합니다.

In [ ]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

## 데이터 세트 그룹 생성 및 생성이 완료될 때까지 기다리기
이제 Amazon Personalize 리소스를 생성해야 합니다. 먼저 도메인이 VIDEO_ON_DEMANDE로 설정된 도메인 데이터 세트 그룹을 만듭니다. 도메인 데이터 세트 그룹은 서로 다른 비즈니스 도메인 및 사용 사례를 지원하는 데이터 세트, 추천 등의 사전 구성된 리소스를 저장하는 컨테이너입니다. 원하는 경우 아래의 이름을 자유롭게 변경해도 됩니다.

### 데이터 세트 그룹 생성

In [ ]:
response = personalize.create_dataset_group(
    name='personalize-video-on-demand-ds-group',
    domain='VIDEO_ON_DEMAND'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

데이터 세트 그룹이 활성 상태가 될 때까지 기다립니다.
아래의 항목에서 데이터 세트 그룹을 사용하려면 먼저 그룹이 활성화되어 있어야 합니다. 아래의 셀을 실행하고 'ACTIVE'로 표시될 때까지 기다립니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 상호 작용 스키마 생성
Personalize가 데이터를 이해하기 위한 핵심 구성 요소는 아래에 정의된 스키마에서 얻어집니다. 이 구성은 Personalize 서비스에 CSV 파일을 통해 제공된 데이터를 읽는 방법을 알려줍니다. 열과 유형은 위에서 만든 파일에 있는 내용과 일치합니다.

In [ ]:
schema = {
  "type": "record",
  "name": "Interactions",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "ITEM_ID",
          "type": "string"
      },
      {
          "name": "EVENT_TYPE",
          "type": "string"
      },
      {
          "name": "TIMESTAMP",
          "type": "long"
      }
  ],
  "version": "1.0"
}

create_interactions_schema_response = personalize.create_schema(
    name='personalize-demo-interactions-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

interactions_schema_arn = create_interactions_schema_response['schemaArn']
print(json.dumps(create_interactions_schema_response, indent=2))

# 항목(영화) 스키마 생성

앞서와 마찬가지로, 항목에 대한 스키마를 만듭니다. 이번에는 영화입니다.

In [ ]:
schema = {
  "type": "record",
  "name": "Items",
  "namespace": "com.amazonaws.personalize.schema",
  "fields": [
    {
      "name": "ITEM_ID",
      "type": "string"
    },
    {
      "name": "GENRES",
      "type": [
        "string"
      ],
      "categorical": True
    },
    {
      "name": "YEAR",
      "type": [
        "string"
      ],
      "categorical": True
    }, 
    {
      "name": "CREATION_TIMESTAMP",
      "type": "long"
    }
  ],
  "version": "1.0"
}
create_items_schema_response = personalize.create_schema(
    name='personalize-demo-items-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

items_schema_arn = create_items_schema_response['schemaArn']
print(json.dumps(create_items_schema_response, indent=2))

# 사용자 스키마 생성

이제 사용자 데이터에 대한 스키마를 만듭니다.

In [ ]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
      {
          "name": "USER_ID",
          "type": "string"
      },
      {
          "name": "GENDER",
          "type": "string",
          "categorical": True
      }
    ],
    "version": "1.0"
}
create_users_schema_response = personalize.create_schema(
    name='personalize-demo-users-schema',
    schema=json.dumps(schema),
    domain='VIDEO_ON_DEMAND'
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))

## 데이터 세트 생성
다음으로 수행할 작업은 방금 만든 스키마를 사용하여 도메인 데이터 세트 그룹 내에 실제 Amazon Personalize 데이터 세트를 만드는 것입니다. 이 데이터 세트에는 훈련을 위한 데이터가 저장됩니다.

### 상호 작용 데이터 세트 생성

In [ ]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interactions_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### 항목 데이터 세트 생성

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### 사용자 데이터 세트 생성

In [ ]:
dataset_type = "USERS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-demo-users",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Personalize 역할 생성
Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. 아래 코드 줄은 그러한 권한을 부여합니다.

참고: 이 노트북에서 코드를 실행하는 데 사용 중인 역할에 새 역할을 생성하는 데 필요한 권한이 있는지 확인합니다. 잘 되지 않을 경우 [Amazon Personalize 개발자 가이드](https://docs.aws.amazon.com/personalize/latest/dg/aws-personalize-set-up-permissions.html)를 참조하세요.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleVODDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)


## 데이터 가져오기
앞서 Magic Movie Machine 데이터를 저장할 도메인 데이터 세트 그룹과 데이터 세트를 만들었습니다. 이제 S3에서 Amazon Personalize로 데이터를 로드하여 영화 추천 시스템을 구축하는 가져오기 작업을 실행합니다.
### 상호 작용 데이터 세트 가져오기 작업 생성

In [ ]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-interactions",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

### 항목 데이터 세트 가져오기 작업 생성

In [ ]:
create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-items",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, items_filename)
    },
    roleArn = role_arn
)

dataset_items_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))

### 사용자 데이터 세트 가져오기 작업 생성

In [ ]:
create_users_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import-users",
    datasetArn = users_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_filename)
    },
    roleArn = role_arn
)

dataset_users_import_job_arn = create_users_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_users_dataset_import_job_response, indent=2))

데이터 세트 가져오기 작업이 활성 상태가 될 때까지 기다립니다.
가져오기 작업이 완료되는 데 시간이 걸릴 수 있습니다. 아래에 활성 상태로 표시될 때까지 기다리세요.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Interactions DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_items_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Items DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_users_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("Users DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 추천 시스템 사용 사례 선택

각 도메인마다 사용 사례가 다릅니다. 추천 시스템을 만들 때에는 특정 사용 사례에 맞게 만들어야 하며, 사용 사례마다 추천을 받는 데 있어서 요구 사항이 다릅니다.

Amazon Personalize가 지원하는 모든 사용 사례에 대한 자세한 내용은 [추천 시스템 사용 사례 선택](https://docs.aws.amazon.com/personalize/latest/dg/domain-use-cases.html)을 참조하세요.

In [ ]:
available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND') # See a list of recommenders for the domain. 
if (len(available_recipes["recipes"])==0):
    # This is a workaround to get the recipes in case 'available_recipes["recipes"]'does not retrieve them
    available_recipes = personalize.list_recipes(domain='VIDEO_ON_DEMAND', nextToken=available_recipes["nextToken"])
display(available_recipes["recipes"])
    

여기서는 *More like X* (X와 유사한 다른 상품) 유형의 추천 시스템을 만들어 보겠습니다. *More like X*(X와 유사한 다른 상품)는 Magic Movie Machine에서 보았던 *Hidden gems* 추천을 단순화한 버전으로, 비슷한 영화에 대한 추천을 생성하지만 덜 알려진 영화를 추천하는 것은 아닙니다. 이 사용 사례에서, Amazon Personalize는 `get_recommendations` 호출에 지정된 userId를 기준으로 사용자가 시청한 동영상을 자동으로 필터링합니다.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'more_like_x_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-more-like-x',
  datasetGroupArn = dataset_group_arn
)
recommender_more_like_x_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

이제 "Top picks for you (고객님을 위한 탑 픽)" 유형의 두 번째 추천 시스템을 만들어 보겠습니다. 이 유형의 추천 시스템은 지정한 사용자를 대상으로 개인화된 스트리밍 콘텐츠 추천을 제공합니다. 이 사용 사례에서, Amazon Personalize는 `Watch` 이벤트에 지정한 userId를 기준으로 사용자가 시청한 동영상을 자동으로 필터링합니다.

In [ ]:
create_recommender_response = personalize.create_recommender(
  name = 'top_picks_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-vod-top-picks',
  datasetGroupArn = dataset_group_arn
)
recommender_top_picks_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

추천 시스템 생성이 완료되고 `ACTIVE` 상태가 될 때까지 기다립니다. 추천 시스템의 상태를 주기적으로 확인합니다.

In [ ]:
%%time

max_time = time.time() + 10*60*60 # 10 hours
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_more_like_x_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_more_like_x_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_more_like_x_arn))

    if status == "ACTIVE":
        break
    else:
        print("The More Like X Recommender build is still in progress")
        
    time.sleep(60)
    
while time.time() < max_time:

    version_response = personalize.describe_recommender(
        recommenderArn = recommender_top_picks_arn
    )
    status = version_response["recommender"]["status"]

    if status == "ACTIVE":
        print("Build succeeded for {}".format(recommender_top_picks_arn))
        
    elif status == "CREATE FAILED":
        print("Build failed for {}".format(recommender_top_picks_arn))

    if status == "ACTIVE":
        break
    else:
        print("The Top Pics for You Recommender build is still in progress")
        
    time.sleep(60)

## 추천 받기
이제 영화 추천 시스템을 만들었으므로 사용자에게 제공되는 영화 추천을 살펴보도록 하겠습니다.

In [ ]:
# reading the original data in order to have a dataframe that has both movie_ids 
# and the corresponding titles to make out recommendations easier to read.
items_df = pd.read_csv('./ml-latest-small/movies.csv')
items_df.sample(10)

In [ ]:
def get_movie_by_id(movie_id, movie_df):
    """
    This takes in an movie_id from a recommendation in string format,
    converts it to an int, and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[movie_df["movieId"]==int(movie_id)]['title'].values[0]
    except:
        print (movie_id)
        return "Error obtaining title"

### 'More Like X' (X와 유사한 다른 상품) 영화 추천 받기

More like X (X와 유사한 다른 상품) 추천 시스템은 유사한 영화에 대한 추천을 생성합니다. 영화의 ID를 제공하면 Amazon Personalize가 유사한 영화의 목록을 반환합니다.

In [ ]:
# First pick a user
test_user_id = "1"

# Select a random item
test_item_id = "81847" #Iron Man 59315, Tangled: 81847

# Get recommendations for the user for this item
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_more_like_x_arn,
    userId = test_user_id,
    itemId = test_item_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [get_movie_by_id(test_item_id, items_df)])

pd.options.display.max_rows = 20
display(user_recommendations_df)

### 'Top picks for you' (고객님을 위한 탑 픽) 추천 받기

이제 *Top picks for you* (고객님을 위한 탑 픽) 추천 시스템을 사용하여 사용자에게 개인화된 추천을 받습니다. 먼저 다음 코드를 사용하여 샘플 사용자에 사용자 메타데이터를 추가합니다. 이 유형의 메타데이터를 사용하여 사용자에 대한 인사이트를 확보할 수 있습니다.

In [ ]:
users_data_df = pd.read_csv('./users.csv')

def get_gender_by_id(user_id, user_df):
    """
    This takes in a user_id and then does a lookup in a specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    try:
        return user_df.loc[user_df["USER_ID"]==int(user_id)]['GENDER'].values[0]
    except:
        print (user_id)
        return "Error obtaining title"

다음으로, 사용자를 선택하고 해당 사용자를 위한 영화 추천을 받습니다.

In [ ]:
# First pick a user
test_user_id = "111" # samples users: 55, 75, 76, 111

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = test_user_id,
    numResults = 20
)

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items_df)
    recommendation_list.append(movie)

column_name = test_user_id+" ("+get_gender_by_id(test_user_id, users_data_df)+")"

user_recommendations_df = pd.DataFrame(recommendation_list, columns = [column_name])

pd.options.display.max_rows =20
display(user_recommendations_df)

## 검토
위의 코드를 사용하여 Magic Movie Machine에서처럼 작동하는 추천 시스템을 만들었습니다. 이전 사용자 행동에 따라 영화 추천을 생성하도록 딥 러닝 모델을 훈련했습니다.

각각 *Top picks for you* (고객님을 위한 탑 픽)와 *More like X* (X와 유사한 다른 상품)라는 기본 사용 사례를 지원할 두 가지 추천 시스템을 만들었습니다. 이제 다음으로, 이 코드를 조정하여 다른 추천 시스템을 만들 수 있습니다.

## 다음 노트북에 대한 참고 사항:
다음 노트북에 필요한 몇 가지 값이 있습니다. 아래 셀을 실행하여 `Clean_Up_Resources.ipynb` 노트북에서 사용할 수 있도록 해당 값을 저장하세요.

이렇게 하면 해당 변수에 대해 저장된 데이터를 덮어쓰고, 변수가 이 노트북에 지정된 값으로 설정됩니다. 

In [ ]:
# store for cleanup
%store dataset_group_arn
%store role_name
%store region